Importing data from tinkoff

In [1]:
import tinkoff.invest as ti
from configparser import ConfigParser
import pandas as pd
import datetime, time
import psycopg2
from io import StringIO
import csv

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
today_date = datetime.date.today()
today_date_start = datetime.datetime(today_date.year, today_date.month, today_date.day)

In [3]:
config_parser = ConfigParser()
config_parser.read('../airflow/tinkoff.cfg')
token = config_parser.get('core', 'TOKEN_TINKOFF')

In [4]:
def get_figi(name):
    instruments = client.instruments.futures()
    for instrument in instruments.instruments:
        if instrument.ticker == name:
            break
    return instrument.figi

In [5]:
def calcDirection(open, close):
    if open==close:
        return '='
    if close-open>0:
        return '+'
    if close-open<0:
        return '-'
    else:
        return 'err'

In [6]:
def create_df(candles: [ti.HistoricCandle]):
    df = pd.DataFrame ([{
        'time' : c.time+datetime.timedelta(hours=3),
        'open' : c.open.units+c.open.nano/1000000000,
        'close' : c.close.units+c.close.nano/1000000000,
        'low' : c.low.units+c.low.nano/1000000000,
        'high' : c.high.units+c.high.nano/1000000000,
        'dir' : calcDirection(c.open.units+c.open.nano/1000000000,c.close.units+c.close.nano/1000000000)
    } for c in candles])
    return df

In [10]:
    with ti.Client(token=token) as client:
        figi = get_figi('RIM2')
        r = client.users.get_accounts()
        r = client.market_data.get_candles(
            figi=figi,
            from_=today_date_start - datetime.timedelta(days=300),
            to = today_date_start,
            interval=ti.CandleInterval.CANDLE_INTERVAL_DAY
        )
        df = create_df(r.candles)

In [11]:
df.head()

,time,open,close,low,high,dir
0,2021-06-15 07:00:00+00:00,162440.0,162510.0,162440.0,162510.0,+
1,2021-06-16 07:00:00+00:00,159740.0,159290.0,159290.0,160300.0,-
2,2021-06-17 07:00:00+00:00,157850.0,159950.0,157850.0,159950.0,+
3,2021-06-18 07:00:00+00:00,159090.0,159090.0,159090.0,159090.0,=
4,2021-06-22 07:00:00+00:00,157990.0,157990.0,157990.0,157990.0,=


In [12]:
df.tail()

,time,open,close,low,high,dir
173,2022-04-05 07:00:00+00:00,102500.0,97250.0,93550.0,103200.0,-
174,2022-04-06 07:00:00+00:00,96550.0,99670.0,94400.0,100470.0,+
175,2022-04-07 07:00:00+00:00,99740.0,106000.0,99620.0,106490.0,+
176,2022-04-08 07:00:00+00:00,105710.0,106150.0,104160.0,109500.0,+
177,2022-04-11 07:00:00+00:00,103200.0,101820.0,98000.0,105040.0,-


Importing to DB

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    191 non-null    datetime64[ns, UTC]
 1   open    191 non-null    float64            
 2   close   191 non-null    float64            
 3   low     191 non-null    float64            
 4   high    191 non-null    float64            
 5   dir     191 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(4), object(1)
memory usage: 9.1+ KB


In [14]:
login='trader'
password='income'
host='localhost'
port=5432
dbname='stocks'


In [15]:
DB_URL = f'user={login} password={password} host={host} port={port} dbname={dbname}'

In [16]:
DB_URL

'user=trader password=income host=localhost port=5432 dbname=stocks'

In [17]:
def load_df_to_db(df: pd.DataFrame, table_name:str) -> None:
    buffer= StringIO()
    df.to_csv(buffer, index=False, sep='|', na_rep='NUL', quoting=csv.QUOTE_MINIMAL, header=False, float_format='%.8f', 
              doublequote=False, escapechar='\\')
    buffer.seek(0)
    copy_query = f"""
        COPY {table_name}({','.join(df.columns)})
        FROM STDIN
        DELIMITER '|'
        NULL 'NUL'
    """
    conn = psycopg2.connect(dsn = DB_URL)
    with conn.cursor() as cursor:
        cursor.copy_expert(copy_query,buffer)
        conn.commit()
        conn.close()

In [18]:
load_df_to_db(df,'data')